In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json

In [2]:
client_id = "j2nijytx8x"
client_pw = "D12z0TZrLISsTlZMNaDejzgrvYZnbn4rhdViIoKz"

In [3]:
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='

data = pd.read_excel('cleanhouse.xlsx', usecols='A, B, C',names=['읍면동', '도움센터명칭','주소'])

# 위도경도 찾기
geo_coordi = []
for add in data['주소']:
    add_urlenc = parse.quote(add)
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID',client_id)
    request.add_header('X-NCP-APIGW-API-KEY',client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)
            if 'addresses' in response_body:
                latitude = response_body['addresses'][0]['y']
                longitude = response_body['addresses'][0]['x']
                print("Success!")
            else:
                print("'result' not exist!")
                latitude = None
                longitude  = None
        else:
            print('Response error code : %d' %rescode)
            latitude = None
            longitude = None
            
    geo_coordi.append([latitude, longitude])

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!


# https://blog.naver.com/rising_n_falling/221596375078

In [4]:
np_geo_coordi = np.array(geo_coordi)
pd_geo_coordi = pd.DataFrame({"읍면동":data["읍면동"].values,
                              "도움센터명칭":data["도움센터명칭"].values,
                              "도로명":data['주소'].values,
                              "위도": np_geo_coordi[:,0],
                              "경도": np_geo_coordi[:,1]})

In [5]:
writer = pd.ExcelWriter('clean_h.xlsx')
pd_geo_coordi.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [6]:
import folium

map = folium.Map(location=[33.365210, 126.529101],zoom_start=10)
for clean, la, ln in zip(data["도움센터명칭"], np_geo_coordi[:,0], np_geo_coordi[:,1]):
    m = folium.Marker([la,ln], popup=clean)
    m.add_to(map)
map

In [7]:
map = folium.Map(location=[33.365210, 126.529101],zoom_start=10)
for clean, la, ln in zip(data["도움센터명칭"], np_geo_coordi[:,0], np_geo_coordi[:,1]):
    m = folium.CircleMarker([la,ln], radius=10, popup=clean,
                            color='#3186cc',fill_color='#3186cc')
    m.add_to(map)
map

In [8]:
clean_help = pd.read_excel('clean_h.xlsx')
h_clean = clean_help.iloc[:,1:]

In [9]:
people = pd.read_excel('인구수.xlsx')
people.columns
people.rename(columns={'지역':'읍면동'},inplace=True)

In [10]:
df_merge_how_left = pd.merge(h_clean, people,how = 'left')

In [11]:
df_merge_how_left

,읍면동,도움센터명칭,도로명,위도,경도,총인구수,가구수
0,한림읍,한림2리 재활용도움센터,제주시 한림읍 한림남길 12,33.410902,126.268900,25149,10404
1,애월읍,고내리 재활용도움센터,제주시 애월읍 고내로 14,33.463066,126.339124,37299,16098
2,애월읍,하귀1리 재활용도움센터,제주시 애월읍 하귀동남3길 16,33.483626,126.414192,37299,16098
3,애월읍,애월리 재활용도움센터,제주시 애월읍 애월해안로 7,33.465151,126.316517,37299,16098
4,구좌읍,종달리 재활용도움센터,제주시 구좌읍 종달논길 31-10,33.492631,126.901820,16214,7706
5,구좌읍,하도리 재활용도움센터,제주시 구좌읍 문주란로 68,33.519811,126.889644,16214,7706
6,조천읍,조천리 재활용도움센터,제주시 조천읍 조천18길 5-5,33.540051,126.643588,25238,10862
7,조천읍,와산리 재활용도움센터,제주시 조천읍 중산간동로 924,33.491592,126.680302,25238,10862
8,우도면,우도면 재활용도움센터,제주시 우도면 조일길 123-5,33.510969,126.960322,1894,1007
9,이도2동,이도2동 재활용도움센터,제주시 도남로5길 25,33.497622,126.528418,50721,20229


In [38]:
map = folium.Map(location=[33.365210, 126.529101],zoom_start=10)
df = df_merge_how_left["총인구수"]/5000
for i, clean, la, ln in zip(range(55), h_clean["도움센터명칭"], h_clean.iloc[:,3], h_clean.iloc[:,4]):
    folium.CircleMarker([la,ln],
                        radius=df[i],
                        color='#3186cc',
                        fill_color='#3186cc').add_to(map)
map